In [ ]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, LSTM, SimpleRNN
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras import layers, models, optimizers
import keras
from tensorflow.keras.applications import EfficientNetB0, VGG16
from tensorflow.keras import * 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io
from tensorflow.keras.models import Model 
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import sklearn
from tensorflow.keras.metrics import Precision, Recall
# from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess


In [ ]:
train_data_dir = '/kaggle/input/classifying-ad-and-pd-v3/3_cls_2/train'
test_data_dir = '/kaggle/input/classifying-ad-and-pd-v3/3_cls_2/test'
image_size = (224, 224)
image_size2 = (600, 600) # EfficientNetB7
batch_size = 16


In [ ]:
train_datagen = ImageDataGenerator(
        preprocessing_function=effnet_preprocess,
        rotation_range=20,     #Random rotation between 0 and 45
        width_shift_range=0.2,   #% shift
        brightness_range=[0.8, 1.2],      # New: Random brightness
        channel_shift_range=10.,          # New: Channel shift
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='constant', 
        cval=125,
        rescale=1.0/255.0,
        validation_split=0.1  # 20% of the data will be used for validation
    ) 
val_datagen = ImageDataGenerator(
    preprocessing_function=effnet_preprocess,
    validation_split=0.1
)
test_datagen = ImageDataGenerator(preprocessing_function=effnet_preprocess,)

In [ ]:
# Create ImageDataGenerators for training and validation with data augmentation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use the 'training' subset
)

# Load the validation data
validation_generator = val_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use the 'validation' subset
) 


# Load the data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
###################### EfficientNetB7
batch_size = 8 
# Create ImageDataGenerators for training and validation with data augmentation
train_generator2 = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size2,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use the 'training' subset
)

# Load the validation data
validation_generator2 = val_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size2,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use the 'validation' subset
) 


# Load the data
test_generator2 = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size2,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
X_train, y_train = next(train_generator)
X_val, y_val = next(validation_generator)
X_test, y_test = next(test_generator)

In [ ]:
total = 0
for class_name in os.listdir(train_data_dir):
    class_path = os.path.join(train_data_dir, class_name)
    if os.path.isdir(class_path):
        n = len([fname for fname in os.listdir(class_path)
                 if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
        print(f"{class_name}: {n} images")
        total += n
print(f"Total images before augmentation: {total}")

In [ ]:
print(f"Train images used (after augmentation): {train_generator.samples}")

## Note: new and old functions below 

I created a new **"build_and_train_model()"** function but also left the old one (just extended the name with "_old") below. The new function does the 2 step process of training and also freezes 80% of the layers of each model - allowing you to fine-tune 20% of the layers. You could always change this to get more specific with each model, but this is a solid generalized approach that I think works. 

In [ ]:
initial_epochs=10
def build_and_train_model_two_phase(
    model_name,
    base_model,
    train_generator,
    validation_generator,
    initial_epochs=initial_epochs,      # train head for 5–10 epochs
    fine_tune_epochs=80,   # then fine-tune for approx 10–15 more (or even more than that?)
    initial_lr=1e-4,
    fine_tune_lr=3e-5,
    dropout_rate=0.7
):
    """
    Two-phase training:
      Phase 1: Freeze ~80% of base_model, train only head for initial_epochs.
      Phase 2: Unfreeze last blocks of base_model, recompile with lower LR, continue training.
    Returns: (fine-tuned_model, combined_history)
    """

    # -----------------------------------------------------------
    # STEP A: Freeze most of the base_model for Phase 1
    # -----------------------------------------------------------
    # 1) freeze the earliest 60 % of layers in base_model.
    N = int(len(base_model.layers) * 0.6)  # a rough t0% cutoff by layer COUNT
    for layer in base_model.layers[:N]:
        layer.trainable = False

    # 2) Unfreeze the remainder of the layers for Phase 1 (so the head can be trained on them, if desired).
    for layer in base_model.layers[N:]:
        layer.trainable = True

    # now - base_model has 80% of its layers frozen, 20% trainable!

    # -----------------------------------------------------------
    # STEP B: Build & compile the full model (head + base)
    # -----------------------------------------------------------
    x = Flatten()(base_model.output)                        # Flatten the feature map
    x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)                    # One hidden Dense layer
    x = Dropout(dropout_rate)(x)                            # Dropout for regularization
    output = Dense(3, activation='softmax')(x)              # 3-way softmax for AD/PD/Control

    model = Model(inputs=base_model.input, outputs=output)

    # Compile with a “standard” learning rate for the head (Phase 1)
    model.compile(
        optimizer=Adam(learning_rate=initial_lr),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )

    # -----------------------------------------------------------
    # STEP C: Compute class_weights 
    # -----------------------------------------------------------
    classes = train_generator.classes
    unique_classes = np.unique(classes)  # e.g. array([0,1,2])

    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=unique_classes,
        y=classes
    )
    class_weights = dict(zip(unique_classes, weights))

    # -----------------------------------------------------------
    # STEP D: Phase 1 — Train only the head (base_model is mostly frozen)
    # -----------------------------------------------------------
   # Learning rate reducer
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=2,
        min_lr=1e-7,
        verbose=2
    )

    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=12,    
        restore_best_weights=True,
        verbose=2
    )
    if (model_name != 'EfficientNetB7'): 
        print(">>> PHASE 1: Training head (with ~80% of base frozen) <<<")
        history_phase1 = model.fit(
            train_generator,
            steps_per_epoch = train_generator.samples // train_generator.batch_size,
            validation_data   = validation_generator,
            validation_steps  = validation_generator.samples // validation_generator.batch_size,
            epochs            = initial_epochs,
            callbacks         = [reduce_lr, early_stop],
            class_weight      = class_weights,
            verbose           = 2
        )
    else:
        print(">>> PHASE 1: Training head (with ~80% of base frozen) <<<")
        history_phase1 = model.fit(
            train_generator2,
            steps_per_epoch = train_generator2.samples // train_generator2.batch_size,
            validation_data   = validation_generator2,
            validation_steps  = validation_generator2.samples // validation_generator2.batch_size,
            epochs            = initial_epochs,
            callbacks         = [reduce_lr, early_stop],
            class_weight      = class_weights,
            verbose           = 2
        )

    # -----------------------------------------------------------
    # STEP E: Phase 2 — Unfreeze the final blocks of base_model
    # -----------------------------------------------------------
    print("\n>>> PHASE 2: Fine-tuning top layers <<<")
    # (a) Freeze everything again first, then unfreeze only the final blocks by name/index.
    for layer in base_model.layers:
        layer.trainable = False

    # ** Here is where you unfreeze only the “last blocks” (approx 20% of layers) **
    # You can either do: 
    #   (1) Replace the condition below with the exact block‐name rule for your architecture.
    #   For example, if this is ResNet50, you might do:
    #       if layer.name.startswith('conv5_') or layer.name.startswith('conv4_'): layer.trainable=True
    #
    #   -or- 
    #   (2) for simplicity - we’ll unfreeze everything from index N onward again:
    for layer in base_model.layers[N:]:
        layer.trainable = True

    # (b) recompile with a lower learning rate (to avoid wrecking the pretrained weights)
    model.compile(
        optimizer=Adam(learning_rate=fine_tune_lr),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )

    # (c) continue training (starting from where Phase 1 left off) - with same class_weight
    if (model_name != 'EfficientNetB7'):
        history_phase2 = model.fit(
            train_generator,
            steps_per_epoch = train_generator.samples // train_generator.batch_size,
            validation_data = validation_generator,
            validation_steps = validation_generator.samples // validation_generator.batch_size,
            epochs = initial_epochs + fine_tune_epochs,
            initial_epoch = initial_epochs,  # resume from Phase 1
            callbacks = [reduce_lr, early_stop],
            class_weight = class_weights,
            verbose = 2
        )
    else: 
        history_phase2 = model.fit(
            train_generator2,
            steps_per_epoch = train_generator2.samples // train_generator2.batch_size,
            validation_data = validation_generator2,
            validation_steps = validation_generator2.samples // validation_generator2.batch_size,
            epochs = initial_epochs + fine_tune_epochs,
            initial_epoch = initial_epochs,  # resume from Phase 1
            callbacks = [reduce_lr, early_stop],
            class_weight = class_weights,
            verbose = 2
        )

    # -----------------------------------------------------------
    # STEP F: Combine the histories so we can plot later if desired
    # -----------------------------------------------------------
    combined_history = {
        'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
        'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
        'accuracy': history_phase1.history.get('accuracy', []) + history_phase2.history.get('accuracy', []),
        'val_accuracy': history_phase1.history.get('val_accuracy', []) + history_phase2.history.get('val_accuracy', []),
        'precision': history_phase1.history.get('precision', []) + history_phase2.history.get('precision', []),
        'val_precision': history_phase1.history.get('val_precision', []) + history_phase2.history.get('val_precision', []),
        'recall': history_phase1.history.get('recall', []) + history_phase2.history.get('recall', []),
        'val_recall': history_phase1.history.get('val_recall', []) + history_phase2.history.get('val_recall', []),
    }

    return model, combined_history

In [ ]:
def build_and_train_model_old(base_model, train_generator, validation_generator, epochs=50,learning_rate=0.0001, dropout_rate=0.5):
    # Freeze the convolutional base
    # base_model.trainable = False
    # for layer in base_model.layers:
    #     layer.trainable = False 
    N = 3
    for layer in base_model.layers[:N]:
        layer.trainable = False
    for layer in base_model.layers[N:]:
        layer.trainable = True

    # Add custom classification layers
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(3, activation='softmax')(x)  # 3 classes: Alzheimer's, Parkinson's, Control

    # Create the model
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile the model
    learning_rate1 = float(1e-4)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate1),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
     

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',    # or 'val_accuracy'
        factor=0.5,            # reduce by half
        patience=3,            # wait this many epochs with no improvement
        min_lr=1e-7,           # never lower than this
        verbose=1
    )

    from sklearn.utils import class_weight
    import numpy as np
    

    classes = train_generator.classes
    class_labels = np.unique(classes)
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=class_labels,
        y=classes
    )
    class_weights = dict(zip(class_labels, weights))
    
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_steps=validation_generator.samples // batch_size,
        validation_data=validation_generator,
        epochs=epochs,
        callbacks=[reduce_lr],
        class_weight=class_weights   # <-- singular
    )
    return model, history


In [ ]:
from tensorflow.keras.callbacks import Callback

class TerminateOnMetric(Callback):
    def __init__(self, monitor='accuracy', threshold=0.80):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        metric_value = logs.get(self.monitor)
        if metric_value is not None:
            if metric_value >= self.threshold:
                print(f"\nReached {self.monitor} = {metric_value:.4f}, stopping training!")
                self.model.stop_training = True

# Example usage:
callback = TerminateOnMetric(monitor='accuracy', threshold=0.80)
 

In [ ]:
 pretrained_models = {
    'VGG16': VGG16(include_top=False, input_shape=(224, 224, 3), weights='imagenet', pooling='avg'),
    'ResNet50': ResNet50(include_top=False, input_shape=(224, 224, 3), weights='imagenet', pooling='avg'),
    'InceptionV3': InceptionV3(include_top=False, input_shape=(224, 224, 3), weights='imagenet', pooling='avg'),
    'Xception': Xception(include_top=False, input_shape=(224, 224, 3), weights='imagenet', pooling='avg'),
    'EfficientNetB0': EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet', pooling='avg'),
    'EfficientNetB7': EfficientNetB7(include_top=False, input_shape=(600, 600, 3), weights='imagenet', pooling='avg'),
    
}

# Store the trained models and their histories
trained_models = {}
histories = {}

for model_name, base_model in pretrained_models.items():
    print(f"Training model: {model_name}")
    model, history = build_and_train_model_two_phase(model_name, base_model, train_generator, validation_generator)
    trained_models[model_name] = model
    histories[model_name] = history


In [ ]:
 def evaluate_model(model, test_generator):
    # Evaluate the model
    test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
    test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall + 1e-2)
    
    # Predict the classes for validation data
    test_preds = model.predict(test_generator)
    test_preds = np.argmax(test_preds, axis=1)
    test_true = test_generator.classes
    
    # Generate confusion matrix
    cm = confusion_matrix(test_true, test_preds)
    
    return test_accuracy, test_precision, test_recall, test_f1, cm

results = []
test_accuracies = []
for model_name, model in trained_models.items():
    print(f"Evaluating model: {model_name}")
    test_accuracy, test_precision, test_recall, test_f1, cm = evaluate_model(model, validation_generator)
    results.append({
        'Model': model_name,
        'Accuracy': test_accuracy,
        'Precision': test_precision,
        'Recall': test_recall,
        'F1 Score': test_f1,
        'Confusion Matrix': cm
    })
    test_accuracies.append(test_accuracy)

print(results)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Example: replace with your actual class order
class_names = ['AD', 'CONTROL', 'PD']

def generate_saliency_map(model, img_path, class_idx, image_size=(224, 224)):
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image

    import tensorflow as tf
    with tf.GradientTape() as tape:
        inputs = tf.cast(img_array, tf.float32)
        tape.watch(inputs)
        predictions = model(inputs)
        loss = predictions[:, class_idx]
    grads = tape.gradient(loss, inputs)
    abs_grads = tf.abs(grads)
    saliency = tf.reduce_max(abs_grads, axis=-1)
    saliency = saliency[0]
    saliency = (saliency - tf.reduce_min(saliency)) / (tf.reduce_max(saliency) - tf.reduce_min(saliency) + 1e-10)
    return saliency

def plot_saliency_map(img_path, saliency, predicted_class_name):
    img = load_img(img_path)
    img = np.array(img)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f'Original Image\nPredicted: {predicted_class_name}')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(saliency, cmap='hot')
    plt.title('Saliency Map')
    plt.axis('off')
    plt.show()

# Example test images (replace with your own paths)
test_images = [
    '/kaggle/input/classifying-ad-and-pd-v3/3_cls_2/train/AD/AD_10.png',
    '/kaggle/input/classifying-ad-and-pd-v3/3_cls_2/train/CONTROL/CONTROLAD_10.png',
    '/kaggle/input/classifying-ad-and-pd-v3/3_cls_2/train/PD/PD_10.png'
]

image_size = (224, 224)  # or your input size

for model_name, model in trained_models.items():
    print(f"\n=== Model: {model_name} ===")
    for img_path in test_images:
        img_arr = img_to_array(load_img(img_path, target_size=image_size)).reshape(1, 224, 224, 3) / 255.0
        preds = model.predict(img_arr)
        predicted_class_idx = np.argmax(preds[0])
        predicted_class_name = class_names[predicted_class_idx]
        print(f"Image: {img_path}")
        print(f"Predicted class: {predicted_class_name} (index: {predicted_class_idx}), Probabilities: {preds[0]}")
        saliency = generate_saliency_map(model, img_path, predicted_class_idx, image_size=image_size)
        plot_saliency_map(img_path, saliency, predicted_class_name)

In [ ]:
import matplotlib.pyplot as plt

def plot_each_model_accuracy(histories, initial_epochs=None):
    """
    Plots training and validation accuracy for each model in separate figures.

    Args:
        histories (dict): Keys are model names, values are Keras History.history dicts.
        initial_epochs (int or None): If provided, draws a vertical line at this epoch.
    """
    for model_name, history in histories.items():
        acc = history.get('accuracy') or history.get('acc')
        val_acc = history.get('val_accuracy') or history.get('val_acc')
        epochs = range(1, len(acc) + 1)

        plt.figure(figsize=(8, 5))
        plt.plot(epochs, acc, label="Train", color='tab:blue', linestyle='-')
        plt.plot(epochs, val_acc, label="Val", color='tab:orange', linestyle='--')
        if initial_epochs is not None:
            plt.axvline(x=initial_epochs, linestyle=':', color='red', label='Layers Frozen/Unfrozen')
        plt.title(f"{model_name} - Training and Validation Accuracy")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Usage:
plot_each_model_accuracy(histories, initial_epochs=initial_epochs)
# If you don't have initial_epochs, just call: plot_each_model_accuracy(histories)

In [ ]:
import matplotlib.pyplot as plt

# Example model names in order
model_names = ['VGG16', 'ResNet50', 'InceptionV3', 'Xception', 'EfficientNetB0']

# Your test_accuracies list
# test_accuracies = [0.81, 0.85, 0.83, 0.80, 0.88]  # Example

# Find the index (and name) of the model with the highest accuracy
best_index = max(range(len(test_accuracies)), key=lambda i: test_accuracies[i])
best_accuracy = test_accuracies[best_index]
best_model = model_names[best_index]
print(f"Highest test accuracy: {best_accuracy:.4f} ({best_model})")

# Plot
plt.figure(figsize=(8, 5))
bars = plt.bar(model_names, test_accuracies, color='limegreen')
plt.ylabel("Test Accuracy")
plt.ylim(0, 1.05)
plt.title("Test Accuracy for Each Pretrained Model")
for bar, acc in zip(bars, test_accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{acc:.2%}", 
             ha='center', va='bottom', fontsize=10)
plt.show()